In [ ]:
import tensorflow as tf
import keras
import keras.backend as K
tf.__version__

import os, sys
import shutil
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import cv2
import pathlib
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.manifold import TSNE
import skimage
import scipy
import copy
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

### Hyperparameters

In [ ]:
BATCH_SIZE = 64
EPOCHS = 25

latent_dim = 1024
num_classes = 18

RANDOM_SEED = 101
random.seed(RANDOM_SEED)
AUTOTUNE = tf.data.AUTOTUNE

val_size = BATCH_SIZE*16
test_size = num_classes*25

input_folder = r"" # Set to GEE Imagery Folder on local computer
weights_folder = r"" # Folder where model weights should be saved
clustering_folder = r"" # Folder to save clustering results

### Data Generation

In [ ]:
files = [os.path.join(input_folder,item) for item in os.listdir(input_folder) if item.endswith('.tif')]
random.seed(101)
random.shuffle(files)
SHAPE = [256,256,6] #get_smallest_img(files) 
# SHAPE[0], SHAPE[1] = min(SHAPE[0],SHAPE[1]), min(SHAPE[0],SHAPE[1])
print("Smallest image in set:", SHAPE)

image_count = len(list(pathlib.Path(input_folder).glob('*.tif')))
print(f"{image_count} total images found")

In [ ]:
class DataGenerator:
    def __init__(self,files_list,shape,num_classes=0,categorize=False,unsupervised=False,mean=0.5):
        self.files = copy.deepcopy(files_list)
        self.shape = shape
        self.categorize = categorize
        self.unsupervised = unsupervised
        self.mean = mean
        self.num_classes = num_classes
        random.shuffle(self.files)
        
    def __len__(self):
        return len(self.files)
    
    def data_augmentation(self,image):
        steps = keras.Sequential(
                [keras.layers.RandomFlip("horizontal_and_vertical"),
                 keras.layers.RandomRotation(0.3),
                 keras.layers.RandomZoom(0.1),
                 keras.layers.RandomCrop(SHAPE[0], SHAPE[1])])
        return steps(image)
    
    def parse_image(self,filename,mean=0.5):
        try:
            image = skimage.io.imread(filename)[:,:,:self.shape[2]]
            # image = tf.convert_to_tensor(image)
            # image = self.data_augmentation(image)
            image = tf.image.resize_with_crop_or_pad(image,self.shape[0], self.shape[1])
            image = tf.image.per_image_standardization(image)
            if self.mean == 0.5:
                image = tf.math.divide(tf.math.add(image,tf.math.abs(tf.math.reduce_min(image))),tf.math.add(tf.math.reduce_max(image),tf.math.abs(tf.math.reduce_min(image)))+0.00001)
            return image
        except:
            pass

    def __getitem__(self,idx):
        file = self.files[idx]
        image = self.parse_image(file)
        if self.categorize == False:
            if self.unsupervised == False:
                return image
            else:
                return image,image
        else:
            onehot = tf.one_hot(range(self.num_classes), self.num_classes)
            category = int(os.path.split(os.path.split(file)[0])[1].split('_')[1])
            hot_label = onehot[int(category)]
            return image, hot_label

    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            if i == self.__len__()-1:
                self.on_epoch_end()    

    def on_epoch_end(self):
        random.shuffle(self.files)

ds_series = tf.data.Dataset.from_generator(DataGenerator(files,SHAPE,unsupervised=True),
                                           output_shapes=(SHAPE,SHAPE),
                                           output_types=(tf.float32,tf.float32))

In [ ]:
# plt.figure(figsize=(20, 20))
# for i,image in enumerate(ds_series.take(64)):
#     plt.subplot(8, 8, i + 1)
#     plt.imshow(image[0][:,:,:3])
#     plt.axis("off")
# plt.subplots_adjust(hspace=0.01, wspace=0.01)

### Spectral Clustering

In [ ]:
# def parse_image(filename):
#     img = skimage.io.imread(filename)
#     img = tf.convert_to_tensor(img)
#     img = tf.image.per_image_standardization(img)
#     img = tf.image.resize_with_crop_or_pad(img,SHAPE[0], SHAPE[1]).numpy()
#     img = np.divide(np.add(img,np.abs(img.min())),np.add(img.max(),np.abs(img.min()))+0.00001)
#     return img

# singulars = []
# pca = PCA(6)
# for file in tqdm(files):
#     image = parse_image(file) 
#     image = np.reshape(image,(image.shape[0]*image.shape[1],image.shape[2]))
#     singulars.append(pca.fit(image).singular_values_)

In [ ]:
# criteria = (cv2.TERM_CRITERIA_EPS, -1, 1.0)
# compactness, labels, _ = cv2.kmeans(np.stack(singulars), num_classes, None, criteria, 100, cv2.KMEANS_RANDOM_CENTERS)
# average_distance_from_each_label_center = compactness / float(len(files))

In [ ]:
# histo = np.histogram(labels,np.unique(labels))
# plt.bar(histo[1][1:],histo[0]);
# plt.xticks(np.unique(labels));

In [ ]:
# %%capture
# cluster_output_save_path = os.path.join(clustering_folder,"Image Generator spectral cluster")
# for file,label in zip(files,labels):  
#     save_dir = rf'{cluster_output_save_path}\class_{str(label[0])}'
#     if not (os.path.exists(save_dir) and os.path.isdir(save_dir)):
#         os.makedirs(save_dir, exist_ok=True)
#     basename = os.path.basename(file)
#     shutil.copy2(file, rf'{save_dir}\{basename}.tif')

In [ ]:
# num_rows = 10
# plt.figure(figsize=(20, 50))
# for col in range(num_classes):
#     class_folder = os.path.join(cluster_output_save_path,f"class_{col}")
#     class_files = os.listdir(class_folder)
#     random.shuffle(class_files)
#     for i in range(num_rows): 
#         try:
#             img = skimage.io.imread(os.path.join(cluster_output_save_path,f"class_{col}",class_files[i]))[:,:,:3]
#             ax = plt.subplot(num_classes, num_rows, col*num_rows + i + 1)
#             plt.imshow(img,aspect="auto")
#             plt.axis("off")
#         except:
#             pass
# plt.subplots_adjust(hspace=0.1, wspace=0.01)
# # plt.savefig('24_cluster_AE.png')

### Autoencoder

In [ ]:
def create_autoencoder():
    input_img = keras.Input(shape=(SHAPE[0],SHAPE[1],SHAPE[2]))
    # init= tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.02)
    x = keras.layers.Conv2D(64, 5, padding='same',strides=(2,2))(input_img)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    
    for filters in [128,194,256,512]:
        x = keras.layers.Conv2D(filters, (3, 3), strides=(2,2), padding='same')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.LeakyReLU()(x)
        x = keras.layers.Conv2D(filters, (3, 3), strides=(1,1), padding='same')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.LeakyReLU()(x)
    
    shape_before_flattening = K.int_shape(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(units=latent_dim, activation='linear')(x)
    encoder_output = keras.layers.BatchNormalization()(x)
    # x = keras.layers.Dense(np.prod(shape_before_flattening[1:]),activation='tanh')(encoder_output)
    x = keras.layers.Reshape((8,8,16))(x)

    for filters in [512,256,194,128,64]:
        x = keras.layers.Conv2DTranspose(filters, (3, 3), padding='same', use_bias=False, strides=(2,2))(x)
        x = keras.layers.BatchNormalization(momentum=0.5)(x)
        x = keras.layers.LeakyReLU()(x)
    decoded = keras.layers.Conv2D(SHAPE[2], 3, padding='same', activation='sigmoid')(x)
    
    encoder = keras.models.Model(input_img, encoder_output, name='enc')
    autoencoder = keras.models.Model(input_img, decoded, name='AE')
    return encoder, autoencoder

In [ ]:
encoder, autoencoder = create_autoencoder()
# autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer=tf.optimizers.Adam(),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
# model_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=15, min_delta = 0.01, mode='max', restore_best_weights=True)
autoencoder.fit(ds_series.batch(BATCH_SIZE,drop_remainder=True).prefetch(AUTOTUNE), epochs=EPOCHS) #, callbacks=[model_early_stopping])

#### Save and Load Models

In [ ]:
weights_dir = os.path.join(weights_folder,"ae_weights_bce")
if not os.path.isdir(weights_dir):
    os.makedirs(weights_dir, exist_ok=True)
encoder.save(os.path.join(weights_dir,"enc_weights.h5"))
autoencoder.save(os.path.join(weights_dir,"ae_weights.h5"))

In [ ]:
encoder = tf.keras.models.load_model(os.path.join(weights_dir,"enc_weights.h5"))
# ae = tf.keras.models.load_model(os.path.join(weights_dir,"ae_weights.h5"))

#### Get Latent Dimensions

In [ ]:
def parse_image(filename,channels=SHAPE[2]):
    image = skimage.io.imread(filename)[:,:,:channels]
    image = tf.convert_to_tensor(image)
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize_with_crop_or_pad(image,SHAPE[0], SHAPE[1])
    return tf.math.divide(tf.math.add(image,tf.math.abs(tf.math.reduce_min(image))),tf.math.add(tf.math.reduce_max(image),tf.math.abs(tf.math.reduce_min(image)))+0.00001)

def graph_forward(model, x):
    return model(x, training=False)

In [ ]:
latent_vectors = {}
for file in tqdm(random.sample(files,2000)):
    latent_vectors[file] = np.asarray(graph_forward(encoder, np.expand_dims(parse_image(file),0))).reshape((latent_dim,))

#### Visualizing Individual Images

In [ ]:
# images = [x[0] for i,x in enumerate(ds_series.take(10).as_numpy_iterator())]


# fig, ax = plt.subplots(len(images),2,figsize=(10,25))
# for i,img in enumerate(images):
#     ax[i,0].imshow(img[:,:,:3]);
#     pred = ((graph_forward(autoencoder, np.expand_dims(img,0))[0,:,:,:3]).numpy()*255).astype('uint8')
#     ax[i,1].imshow(pred);

In [ ]:
# plt.imshow(parse_image(r""\ # insert path to test image
#                        ,3));
# plt.axis('off')

In [ ]:
# locs = [x for x in range(latent_dim)]
# plt.figure(figsize=(10,1))
# plt.bar(locs,list(graph_forward(encoder, np.expand_dims(parse_image(r"" \ # insert path to test image
#    ),0)).numpy().flatten()));
# plt.axis('off')

#### t-SNE

In [ ]:
# t-SNE
keys = list(latent_vectors.keys())
random.shuffle(keys)
shuffled_vectors = {key: latent_vectors[key] for key in keys}
shuffled_vectors = dict(itertools.islice(shuffled_vectors.items(), 2000))
tsne = TSNE(n_components=2, learning_rate=200, perplexity=50, angle=0.5, verbose=0, init='pca').fit_transform(np.asarray(list(shuffled_vectors.values())))
tx, ty = tsne[:,0], tsne[:,1]
tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

In [ ]:
width = 4000
height = 3000
max_dim = 100

full_image = Image.new('RGBA', (width, height))
for img, x, y in zip(list(shuffled_vectors.keys()), tx, ty):
    tile = Image.fromarray((parse_image(img,3).numpy()*255).astype('uint8'))
    rs = max(1, tile.width/max_dim, tile.height/max_dim)
    tile = tile.resize((int(tile.width/rs), int(tile.height/rs)), Image.ANTIALIAS)
    full_image.paste(tile, (int((width-max_dim)*x), int((height-max_dim)*y)), mask=tile.convert('RGBA'))

plt.figure(figsize = (16,12))
plt.imshow(full_image)

In [ ]:
full_image.save('tSNE_1000_images_latentx3.png')

In [ ]:
!pip install rasterfairy
import rasterfairy

# nx * ny = 1000, the number of images
nx = 50
ny = 40

# assign to grid
grid_assignment = rasterfairy.transformPointCloud2D(tsne, target=(nx, ny))

In [ ]:
tile_width = 72
tile_height = 56

full_width = tile_width * nx
full_height = tile_height * ny
aspect_ratio = float(tile_width) / tile_height

grid_image = Image.new('RGB', (full_width, full_height))

for img, grid_pos in zip(list(shuffled_vectors.keys()), grid_assignment[0]):
    idx_x, idx_y = grid_pos
    x, y = tile_width * idx_x, tile_height * idx_y
    tile = Image.fromarray((parse_image(img,3).numpy()*255).astype('uint8'))
    tile_ar = float(tile.width) / tile.height  # center-crop the tile to match aspect_ratio
    if (tile_ar > aspect_ratio):
        margin = 0.5 * (tile.width - aspect_ratio * tile.height)
        tile = tile.crop((margin, 0, margin + aspect_ratio * tile.height, tile.height))
    else:
        margin = 0.5 * (tile.height - float(tile.width) / aspect_ratio)
        tile = tile.crop((0, margin, tile.width, margin + float(tile.width) / aspect_ratio))
    tile = tile.resize((tile_width, tile_height), Image.ANTIALIAS)
    grid_image.paste(tile, (int(x), int(y)))

plt.figure(figsize = (16,12))
plt.imshow(grid_image)

In [ ]:
grid_image.save('grid_image_2000x3.png')

#### Kmeans Clustering

In [ ]:
def parse_image(filename):
    image = skimage.io.imread(filename)[:,:,:SHAPE[2]]
    image = tf.convert_to_tensor(image)
    image = tf.image.resize_with_crop_or_pad(image,SHAPE[0], SHAPE[1])
    image = tf.image.per_image_standardization(image)
    return tf.math.divide(tf.math.add(image,tf.math.abs(tf.math.reduce_min(image))),tf.math.add(tf.math.reduce_max(image),tf.math.abs(tf.math.reduce_min(image)))+0.00001)

def graph_forward(model, x):
    return model(x, training=False)

latent_vectors = []
for file in tqdm(files):
    latent_vectors.append(np.asarray(graph_forward(encoder, np.expand_dims(parse_image(file),0))).reshape((latent_dim,)))
latent_vectors = np.asarray(latent_vectors)

In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS, -1, 1)
compactness, labels, _ = cv2.kmeans(latent_vectors, num_classes, None, criteria, 100, cv2.KMEANS_PP_CENTERS)
average_distance_from_each_label_center = compactness / float(len(files))
print(average_distance_from_each_label_center)
histo = np.histogram(labels,np.unique(labels))
plt.bar(histo[1][1:],histo[0]);
plt.xticks(np.unique(labels));

In [ ]:
%%capture
cluster_output_save_path = os.path.join(clustering_folder,"Image Generator ww autoencoder bce 18")
for file,label in zip(files,labels):
    save_dir = rf'{cluster_output_save_path}\class_{str(label[0])}'
    if not (os.path.exists(save_dir) and os.path.isdir(save_dir)):
        os.makedirs(save_dir, exist_ok=True)
    basename = os.path.basename(file)
    shutil.copy2(file, rf'{save_dir}\{basename}.tif')

In [ ]:
num_rows = 3
i = 0
plt.figure(figsize=(17, 50))
for col in range(num_classes):
    class_folder = os.path.join(cluster_output_save_path,f"class_{col}")
    class_files = os.listdir(class_folder)
    random.shuffle(class_files)
    for index in range(num_rows): 
        try:
            img = skimage.io.imread(os.path.join(cluster_output_save_path,f"class_{col}",class_files[index]))
            nat = np.stack([img[:,:,0],img[:,:,1],img[:,:,2]],axis=-1)
            cir = np.stack([img[:,:,3],img[:,:,0],img[:,:,1]],axis=-1)
            swir = np.stack([img[:,:,5],img[:,:,4],img[:,:,0]],axis=-1)
            ax = plt.subplot(num_classes, num_rows*3, i + 1)
            plt.imshow(nat,aspect="auto")
            plt.axis("off")
            ax = plt.subplot(num_classes, num_rows*3, i + 2)
            plt.imshow(cir,aspect="auto")
            plt.axis("off")
            ax = plt.subplot(num_classes, num_rows*3, i + 3)
            plt.imshow(swir,aspect="auto")
            plt.axis("off")
            i+=3
        except:
            pass
plt.subplots_adjust(hspace=0.1, wspace=0)
plt.tight_layout()
plt.savefig('30_cluster_AE_combo_5.png')

In [ ]:
num_rows = 13
plt.figure(figsize=(20, 30))
for col in range(num_classes):
    class_folder = os.path.join(cluster_output_save_path,f"class_{col}")
    class_files = os.listdir(class_folder)
    random.shuffle(class_files)
    for i in range(num_rows): 
        try:
            img = skimage.io.imread(os.path.join(cluster_output_save_path,f"class_{col}",class_files[i]))[:,:,:3]
            ax = plt.subplot(num_classes, num_rows, col*num_rows + i + 1)
            plt.imshow(img,aspect="auto")
            plt.axis("off")
        except:
            pass
plt.subplots_adjust(hspace=0.1, wspace=0.01)
plt.savefig('30_cluster_AE_5.png')

In [ ]:
for folder in os.listdir(cluster_output_save_path):
    for file in os.listdir(os.path.join(cluster_output_save_path,folder)):
        filename = os.path.join(cluster_output_save_path,folder,file)
        image = Image.fromarray(skimage.io.imread(filename)[:,:,:3])
        outname = os.path.basename(filename[:-9])
        image.save(os.path.join(cluster_output_save_path,folder,f'{outname}.jpg'))
        os.remove(filename)